# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel. Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Vamos a ver varias formas de representar el lenguaje mediante números: `encoding ordinal`, `one-hot encoding` y `word embedding`

## Encoding ordinal

Esta es la manera más básica de representar numéricamente un lenguaje, y consiste en asignar un número a cada palabra, por ejemplo, podemos decir que gato lo representaremos con un 1, perro con un 2, mesa con un 3, ...

## One-hot encoding

Aunque el encoding ordinal nos resuelve el problema tiene varios problemas

 * En el ejemplo que hemos dado se puede asumir que mesa (3) equivale a gato (1) + perro (2), lo cual no tiene nada que ver. En el lenguaje existen relaciones entre las palabras, por ejemplo perro y perra tienen mucha relación, por lo que es necesario un sistema de codificación mejor
 * Importancia de las palabras. Las palabras van a entrar a redes neuronales, que como hemos visto se componene de capas, con unos pesos, mediante las cuales se realizan unas operaciones matemáticas. Por lo que puede llegar a pasar que la red le de más importancia a las palabras con un número mayor

Debido a esto, se pensó en una alternativa, el one-hot encodding. Aquí lo que se hace es crear vectores de tamaño N, donde cada palabra corresponderá a un vector de todo ceros, menos un uno en una posición determinada. En el ejemplo de antes gato correspondería al vector `[1 0 0 ... 0]`, perro al vector `[0 1 0 ... 0]`, mesa al vector `[0 0 1 ... 0]`, ...

Haciendo esta codificación arreglamos los dos problemas que hemos contados del encoding ordinal y además añadimos una ventaja, como las redes neuronales realizan operaciones matriciales, si a una matriz le multiplicas por un vector de todo ceros, menos un uno en una posición, lo que estás haciendo es que el resultado de la operación es obtener la columna o la fila correspondiente a la posición del 1

![one-hot encodding matrix multiplication](Imagenes/one_hot_encoding_matrix_multiplication.png)

Esto es muy útil si en redes neuronales quieres una fila o una columna de una matriz, porque al no obtener la fila o la columna haciendo slicing, es decir, `matriz[i]`, sino que se obtiene mediante una multiplicación, esta operación es deribable y por tanto se podría añadir al algoritmo del descenso del gradiente

## Word embedding

El one-hot encoding está muy bien, pero crea un nuevo problema, y es que en un lenguaje de un millón de palabras, necesitaríamos que cada palabra se codificase con vectores de 999.999 ceros y 1 solo uno. Esto a la hora de hacer operaciones matriciales no es muy eficiente y hace que las redes y datos ocupen mucha memoria

Para solucionar esto se empezó a usar el hot encoding a secas o también llamado word embedding, que consiste en seguir representando cada palabra en vectores, pero ahora cada vector tendrá un tamaño fijo, de momento digamos N, y en el que cada uno de sus componentes puede tener cualquier valor.

Esto resuelve el problema de la memoria y la ineficiencia en las operaciones matriciales

Además crea una nueva característica, ya que al representar las palabras en vectores de N dimensiones, en realidad lo que estamos haciendo es representar las palabras en un espacio N dimensional. Si esto te suena a muy complicado no te preocupes, que ahora te lo cuento de otra manera , ya verás que sencillo es y cómo lo vas a entender

Supongamos que en vez de ser un espacio de N dimensiones, tenemos un espacio de 2 dimensiones, alto y ancho, pues podríamos representar las palabras de esta manera

![word embedding 2 dimmension](Imagenes/word_embedding_2_dimmension.png)

Como puedes ver todas las palabras que tienen semejanza están juntas. PUes ahora supón que en vez de 2 dimensiones tenemos 3, alto, ancho y profundo, podríamos representar las palabras de esta manera

![word embedding 3 dimmension](Imagenes/word_embedding_3_dimmension.png)

Ahora las palabras siguen estando juntas por semejanza, pero tenemos una dimensión más para poder hacer más grupos.

Como en el lenguaje tenemos muchisimas palabras no nos vale con 3 dimensiones, por lo que tenemos que hacerlo con muchas más. El modelo más grande de word embeding de Bert es de 1024 dimensiones, mientras que el modelo más grande de word embeding de GPT3 es de 4096 dimensiones

Esta posibilidad de poder representar las palabras en grupos por semejanza también nos da otra ventaja, y es la relación entre palabras. Si al vector que representa la palabra `rey` le restas el vector que representa la palabra `hombre` y le sumas el vector que representa la palabra `mujer` obtienes un vector muy parecido al que representa la palabra `reina`.

Como hemos dicho, en el lenguaje existe una gran relación entre las palabras, lo cual va a ser muy importante para entender frases y además es uno de los mecanismos más importantes de los transformers, el de atención, de hecho el paper de los transformers se llama `Attention is all you need` (`Atención es todo lo que necesitas`). Así que gracias a esta forma de representar las palabras podemos prestar atención a esta relación entre palabras

### Como se crean los word embeddings

Hemos explicado cómo se representan muy bien las palabras gracias a los word embedding, hemos mostrado como las palabras con semejanzas están representadas juntas y hemos hablado de los word embeddings de Bert y GPT3. Pero no hemos explicado cómo se crean, es decir, cómo se decide qué valor tiene cada item del vector para cada palabra. Obviamente eso no lo hace ninguna persona

Cuando se entrenan los grandes modelos, también se entrenan las capas de embedding, ya que pertenecen al modelo, por lo que durante el entrenamiento de la red, todos estos vectores que representan las palabras del lenguaje van cogiendo forma y se van uniendo en grupos (cluesters) por significado semántico. Por lo que si en el entrenamiento se ha introducido texto muy variado, este word embeding estará mejor hecho que si el texto no es variado. Por ejemplo, si el dataset solo contiene textos económicos, contables, etc. es muy probable que en el word embedding, la palabra `banco` no tenga nada de relación con la palabra `silla`

Por lo que para un uso general del modelo, es necesario que el dataset de entrenamiento sea lo más variado posible. Pero si el uso va a ser muy específico de ese tema, no es necesario que el dataset sea tan variado

### Implementación

Vamos a usar el embedding de Bert para ver cómo representa las palabras, para ello vamos a usar [huggingface](https://huggingface.co) que se hizo muy popular gracias a su librería [transformers](https://huggingface.co/docs/transformers)

Primero vamos a importar las librerías necesarias

In [2]:
from transformers import BertTokenizer, BertModel
import torch

Cargamos el modelo Bert

In [7]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Ahora su tokenizador

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Creamos el token de la palabra `hola`

In [4]:
# Codifica "hola" en tokens y agrega los tokens especiales [CLS] y [SEP]
input_ids = tokenizer.encode("hola", add_special_tokens=True)

Más adelante entenderás esto, pero el modelo Bert solo contiene la parte de codificación del transformer, por lo que para obtener el embedding de la palabra `hola` lo que hacemos es pasarla por el modelo. Primero convertimos su token a un vector y luego hacemos inferencia

In [5]:
# Convierte los IDs a tensores y pasa por el modelo para obtener los embeddings
input_ids_tensor = torch.tensor([input_ids])
with torch.no_grad():
    outputs = model(input_ids_tensor)

Obtenemos el embedding

In [9]:
# El primer elemento de la salida del modelo son los embeddings para cada token.
# Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
hola_embedding = outputs[0][0][1]

type(hola_embedding), hola_embedding.shape, hola_embedding

(torch.Tensor,
 torch.Size([768]),
 tensor([-2.5825e-01, -4.1451e-01,  1.3310e+00,  5.0903e-02, -1.2352e-01,
         -2.5993e-01, -7.3110e-01, -1.4308e-01,  7.0695e-01,  6.4720e-02,
         -4.4573e-01,  2.7530e-01, -4.5984e-01,  9.7510e-01,  3.5397e-01,
          2.4433e-01,  4.8471e-01,  7.0356e-01, -2.9036e-01,  1.8888e-02,
          7.4378e-01,  5.2929e-01, -7.6937e-01, -3.6225e-01, -2.5520e-01,
         -1.0521e+00, -1.0024e+00, -9.4964e-01, -4.0125e-01, -2.5991e-01,
         -1.7926e-02,  1.9220e-01,  6.0427e-01,  1.9149e-01, -5.8110e-01,
          9.2513e-01, -1.6392e-01, -3.8327e-01,  8.4580e-02, -6.9295e-01,
         -7.2972e-02, -3.5562e-01, -3.9924e-01,  1.2729e-01,  7.2401e-01,
         -7.1348e-01,  1.1409e-01, -9.0983e-02,  2.5221e-01, -6.5058e-01,
          3.4572e-01, -2.4137e-01,  1.7462e-01,  3.0785e-01, -1.0352e-01,
          8.9522e-02,  6.7650e-01, -1.9520e-01, -4.4381e-01,  7.4259e-01,
          3.4907e-02, -2.0905e-01, -2.2943e-01, -2.7157e-01, -5.2984e-02,
   

Como vemos obtenemos un tensor de tamaño 768, esto quiere decir que el modelo Bert que hemos usado tiene un word embedding de tamaño 768, es decir, cada palabra estará representada por un vector de 768 valores distintos

In [12]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity

# Carga el pre-trained BERT
model = BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Lista de palabras
palabras = ["rey", "reina", "hombre", "mujer"]

# Diccionario para guardar los embeddings
embeddings = {}

for palabra in palabras:
    # Codifica la palabra en tokens y agrega los tokens especiales [CLS] y [SEP]
    input_ids = tokenizer.encode(palabra, add_special_tokens=True)

    # Convierte los IDs a tensores y pasa por el modelo para obtener los embeddings
    input_ids_tensor = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids_tensor)

    # El primer elemento de la salida del modelo son los embeddings para cada token.
    # Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
    embeddings[palabra] = outputs[0][0][1]

# Realiza la operación rey - hombre + mujer
resultado = embeddings["rey"] - embeddings["hombre"] + embeddings["mujer"]

# Calcula la similitud coseno entre el resultado y la palabra "reina"
similitud = cosine_similarity(resultado.unsqueeze(0), embeddings["reina"].unsqueeze(0))

print(f"La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: {similitud.item()}")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: 0.728291928768158
